In [1]:
import pyMorfologik
from pyMorfologik import Morfologik
from pyMorfologik.parsing import ListParser

In [2]:
import numpy as np
import pandas as pd
import nltk
import re
import codecs
from copy import deepcopy
from sklearn import feature_extraction
#import mpld3

In [109]:
scraping = pd.read_csv('dane.csv')
#ramka danych z treścią w pierwszej kolumnie i adresami w drugiej

In [110]:
documents = scraping['text']
type(documents)

pandas.core.series.Series

In [111]:
#text after an author's name is not important for further analysis. Regular expression below should remove it. 
#It must be done before removing end of line's marks (/n)  
import re
re.compile('[a-zA-Z]',  re.UNICODE) #compiling regex for non-english characters
#for author's description within a domain's name
articles = [re.sub("\\n+_?[A-Z][a-z]+\s[A-Z][a-z]+\s*\\n?[A-Za-z\s]+\.pl.*(\\n.*)*", " ", str(a)) for a in documents]


In [112]:
def words_num(set):
    words=0
    for a in set:
        words += len(a)
    return words
def set_reduction(set1,set2):
    
    return print('Zbiór zmniejszył się o: {} słów. Wynosi: {:.5f} początkowego '
                 .format((words_num(set1) - words_num(set2)), (words_num(set2)/words_num(set1)))) 
set_reduction(documents, articles)

Zbiór zmniejszył się o: 228563 słów. Wynosi: 0.91073 początkowego 


In [113]:
#for author's description without a domain name 
articles = [re.sub("\\n+_?[A-Z]\w+\s[A-Z]\.?\s[A-Z]\w+\s*\\n\\n.*", " ", str(a)) for a in articles]


In [114]:
set_reduction(documents, articles)

Zbiór zmniejszył się o: 228669 słów. Wynosi: 0.91069 początkowego 


In [115]:
#removing local and global urls 
articles = [re.sub("/[a-zA-Z0-9_\-\./]+|htt[^\s]*", " " , str(a)) for a in articles ]

In [116]:
set_reduction(documents, articles)

Zbiór zmniejszył się o: 255920 słów. Wynosi: 0.90005 początkowego 


In [117]:
#clearing articles from special characters and punctualtions   

articles = [re.sub("\\n", " ", str(a)) for a in articles]
articles = [re.sub("[\*|\.|,|:|;|\?|!|#|\(|\)|-]", " ", str(a)) for a in articles]


In [118]:
set_reduction(documents, articles)

Zbiór zmniejszył się o: 255920 słów. Wynosi: 0.90005 początkowego 


In [119]:
#for articles without author deleting the last part of the article containing dates, comments and autopromotional texts

articles2 = [re.sub("data utworzenia.*|Komentarze.*", "", str(a)) for a in articles]

In [120]:
set_reduction(documents, articles2)

Zbiór zmniejszył się o: 660910 słów. Wynosi: 0.74188 początkowego 


In [121]:
reduction=[]
for a in articles:
    m = re.search("(data utworzenia.*|Komentarze.*)", str(a))
    if m:
        found = m.group(1)
        reduction.append(found)
reduction[:5]    

['Komentarze  Brak komentarzy  Dodaj swój komentarz  Autor   Temat   Treść komentarza    []         Jak obniżyć podatek za 2017 rok       PIT 2017   aktualne ulgi i terminy                                Pomoc on line      [e doradca PIT pl]       Jeżeli   masz problemy z fiskusem     możesz także skorzystać   z pomocy profesjonalnego doradcy podatkowego   \\  zadając pytanie lub przedstawiając swoje wątpliwości  www e doradca PIT pl  Jeżeli trafiłeś tu w celu rozliczenia Twojego PITa za ostatni rok  zapraszamy także na aktualny bezpłatny poradnik w zakresie rozliczeń za rok 2007   Twoje pytanie  ',
 'Komentarze  Brak komentarzy  Dodaj swój komentarz  Autor   Temat   Treść komentarza    []         Jak obniżyć podatek za 2017 rok       PIT 2017   aktualne ulgi i terminy                         ',
 'Komentarze    Autor    aneta _ 12 03 2018 21 32 18 _    Temat    ulga  syn uzyskał w 2017 przychód 5091  po odliczeniu kosztów i składek zus wychodzi dochodu 3058 02zł czy przysługuje jeszcze

In [122]:
articles = [re.sub("data utworzenia.*|Komentarze.*", "", str(a)) for a in articles]

## Stemming

In [123]:
#stopwords = nltk.corpus.stopwords.words('english')
stopwords = pd.read_csv("stopwords-pl.txt", header=None)

In [124]:
type(stopwords)
stopwords = stopwords.iloc[:,0]
stopwords = stopwords.tolist()

In [125]:
parser = ListParser()
stemmer = Morfologik()
articles = [a.lower() for a in articles]
articles = [re.sub("\d+", "", str(a)) for a in articles]
articles = [nltk.word_tokenize(a, language='polish') for a in articles]
articles = [[w for w in a if w not in stopwords] for a in articles]


     

In [126]:
from collections import Counter
def counting(set):
    c= Counter(set)
    aCounted={}
    for i in set:
        if i not in aCounted:
            aCounted[i]=c[i]

    sort_count = sorted(aCounted.items(), key=lambda item: item[1], reverse=True)

    return pd.DataFrame(sort_count)


In [127]:
articles_stemmed = [stemmer.stem(a, parser) for a in articles]

In [128]:
#stemmming is a long process. If something goes wrong, it'll be better to have a copy :)
x_l=deepcopy(articles_stemmed)

In [129]:
unamb_corpora=[]
for a in x_l:
    unamb=[]
    for i in range(len(a)):
        if len(a[i][1])==1:
            unamb+=a[i][1]
        elif len(a[i][1])==0:
            unamb+=a[i][0]
    unamb_corpora+=unamb #full match for all corpora      

## Dividing set for ambigous and nonambigous words

In [130]:
stemmed_arts=[]
document_other=[] #remaining words, without a counterpart in unambiguous
document_final=[]
for a in x_l:
    
    amb=[] #ambiguous
    unamb=[] #unambiguous
    for i in range(len(a)):
    
        if len(a[i][1])==1:
            unamb+=a[i][1]
        elif len(a[i][1])==0:
            unamb+=a[i][0]
        else:
            amb.append(a[i]) 
    
    amb_dict={} #making dictionary with values in a list

    for i in amb:
        amb_dict[i[0]]=[]
        for j in i[1]:
            amb_dict[i[0]].append(j)
    
    
    for i in amb_dict:
        for j in range(len(amb_dict[i])):
            if amb_dict[i][j] in unamb:
                #print(amb_dict[i][j])
                unamb.append(amb_dict[i][j])
            elif amb_dict[i][j] in unamb_corpora:
                #print(amb_dict[i][j])
                unamb.append(amb_dict[i][j])
    else:
        #document_other.append(amb_dict[i][0]) #when we haven't proper assignment
        document_other.append(i) #when we haven't proper assignment

    document_final=unamb+document_other
        #pd_document = counting(document_final)
    stemmed_arts.append(document_final)

In [131]:
stemmed_arts[1]

['archiwum',
 'roczny',
 'zależność',
 'rodzaj',
 'osiągnąć',
 'dochód',
 'podatnik',
 'móc',
 'rok',
 'trzy',
 'zeznanie',
 'podatkowy',
 'należeć',
 'rozważnie',
 'wybrać',
 'formularz',
 'załącznik',
 'związać',
 'ulga',
 'odliczenie',
 'należeć',
 'złożyć',
 'rok',
 'pita',
 'l',
 'pita',
 'styczeń',
 'rok',
 'pita',
 'chcieć',
 'złożyć',
 'korekta',
 'móc',
 'archiwalny',
 'formularz',
 'pita',
 'program',
 'komputerowy',
 'dochód',
 'odliczenie',
 'podlegać',
 'podatkowy',
 'podstawa',
 'podatek',
 'złoty',
 'podatek',
 'wynosić',
 '’',
 'złoty',
 'kwota',
 'zmniejszać',
 'podatek',
 'złoty',
 '’',
 'złoty',
 '’',
 'złoty',
 '’',
 'złoty',
 '+',
 'nadwyżka',
 '’',
 'złoty',
 '’',
 'złoty',
 '’',
 'złoty',
 '+',
 'nadwyżka',
 '’',
 'złoty',
 'dochód',
 'odliczyć',
 'dochód',
 'rodzaj',
 'ulga',
 'prywatny',
 'internet',
 'złoty',
 'ulga',
 'rehabilitacyjny',
 'opłacić',
 'przewodnik',
 '’',
 'złoty',
 'pies',
 'przewodnik',
 '’',
 'złoty',
 'lek',
 'zalecić',
 'specjalista',
 'nad

In [132]:
docs =[]
for a in stemmed_arts:
    b = " ".join(a)
    docs.append(b)


In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000

#tfidf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=stopwords)
tfidf = tfidf_vectorizer.fit_transform(docs)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

#lda
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=stopwords)
tf = tf_vectorizer.fit_transform(docs)
tf_feature_names = tf_vectorizer.get_feature_names()


/home/kodolamacz/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [134]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics, 
          random_state=1, alpha=.1, l1_ratio=.5, 
          init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(
    n_topics=no_topics, max_iter=5, learning_method='online',
    learning_offset=50.,random_state=0).fit(tf)

In [135]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
wyłączenia świadczenia życie podatkowy odliczeniu rady warto ustawa formalności dzień
Topic 1:
nieruchomość podatek lokal budynek mieszkalny sprzedaż wartość grunt nabyć wybudować
Topic 2:
pita deklaracja rok podatnik podatek ulga płatnik podatkowy druk roczny
Topic 3:
złoty kwartał kwota przeciętny nauka euro uczeń rok wskaźnik kurs
Topic 4:
data zmiana podatek ustawa nowy projekt rok finanse podatkowy ministerstwo
Topic 5:
działalność gospodarczy przedsiębiorca prowadzić firma nipa wykonywać podatnik wpis spółka
Topic 6:
polski podatek praca kraj granica dochód rezydent zasada zagraniczny terytorium
Topic 7:
darowizna spadek rodzeństwo majątkowy nabywca rzecz złoty grupa wartość osoba
Topic 8:
składka praca pracownik minimalny społeczny pracodawca emerytura zdrowotny płatnik ubezpieczenie
Topic 9:
spis natura wartość składnik cena majątek remanent niematerialny początkowy sporządzić
Topic 0:
leśny podatek las grunt podatkowy budynek sprzedaż odliczeniu obowiązek dzień
Topic 

In [136]:
doc_topic = lda.transform(tf)
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    print("doc: {} topic: {}\n".format(n,topic_most_pr))
    print(scraping.iloc[n,1])


doc: 0 topic: 8

http://www.pit.pl/pit-2017/
doc: 1 topic: 8

http://www.pit.pl/2005/
doc: 2 topic: 8

http://www.pit.pl/roczne2003/
doc: 3 topic: 8

http://www.pit.pl/rozliczenia-roczne-pit-20132014--deklaracje-pit--ulgi-podatkowe-i-odliczenia-20132014--pitpl-12831/
doc: 4 topic: 7

http://www.pit.pl/transakcje-walutowe--inwestycje--podatki-w-praktyce-13694/
doc: 5 topic: 8

http://www.pit.pl/pity_2014_w_2015_za_rok_2015_pity_roczne_2015_rozliczenie_pit_2014_pit_20014/
doc: 6 topic: 5

http://www.pit.pl/rodzina--odpowiedzialnosc-osobista--podatki-w-praktyce-13691/
doc: 7 topic: 8

http://www.pit.pl/pity-roczne/pity-2015-w-2016-za-rok-2015--pity-roczne-2015-rozliczenie-pit-2015--pity-ulgi-odliczenia-14934/
doc: 8 topic: 7

http://www.pit.pl/sprzedaje-za-granica--podatki-a-handel-w-internecie-11353/
doc: 9 topic: 7

http://www.pit.pl/koszty-przesylki--podatki-w-internecie-10257/
doc: 10 topic: 7

http://www.pit.pl/zbycie-praw-autorskich-w-sieci--prywatnie--podatki-a-handel-w-internecie-

In [97]:
from scipy.cluster.hierarchy import ward, dendrogram

linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances

fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(linkage_matrix, orientation="right", labels=titles);

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout

#uncomment below to save figure
#plt.savefig('ward_clusters.png', dpi=200) #save figure as ward_clusters

NameError: name 'dist' is not defined